In [12]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import re
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from joblib import dump

In [2]:
# URL del conjunto de datos
url_dataset = "https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv"

# Cargar el conjunto de datos en un DataFrame de pandas
df = pd.read_csv(url_dataset)

# Visualizar las primeras filas del DataFrame
print(df.head())

                                                 url  is_spam
0  https://briefingday.us8.list-manage.com/unsubs...     True
1                             https://www.hvper.com/     True
2                 https://briefingday.com/m/v4n3i4f3     True
3   https://briefingday.com/n/20200618/m#commentform    False
4                        https://briefingday.com/fan     True


In [6]:
# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Preprocesamiento de URLs

def preprocess_url(url):
    url_parts = re.findall(r'\b\w+\b', url)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    processed_url = [lemmatizer.lemmatize(word) for word in url_parts if word.lower() not in stop_words]
    return processed_url

df['processed_url'] = df['url'].apply(preprocess_url)

# División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['processed_url'], df['is_spam'], test_size=0.2, random_state=42)

# Visualizar las primeras filas del conjunto de entrenamiento y prueba
print("Conjunto de entrenamiento:")
print(X_train.head())
print("\nConjunto de prueba:")
print(X_test.head())

Conjunto de entrenamiento:
1569    [http, www, morningbrew, com, daily, story, 20...
2229    [http, www, morningbrew, com, daily, story, 20...
2296    [http, www, nytimes, com, article, maskne, acn...
1800    [http, podcasts, apple, com, u, podcast, foxy,...
1273    [http, www, nycpride, org, event, nyc, pride, ...
Name: processed_url, dtype: object

Conjunto de prueba:
1376      [http, link, morningbrew, com, manage, 5z8, oc]
932     [http, www, sciencedaily, com, release, 2019, ...
144     [http, www, ft, com, content, eae603a4, a369, ...
1752                        [http, thehustle, co, career]
51      [http, www, caltech, edu, news, natural, fluid...
Name: processed_url, dtype: object


In [19]:
# Define una función para tokenizar las URLs
def tokenizar_url(url):
    return url

# Crear un pipeline con un vectorizador CountVectorizer y un clasificador SVM
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(analyzer=lambda x: x)), # Convertir la lista de tokens en una cadena para CountVectorizer
    ('classifier', SVC()) # Clasificador SVM con parámetros por defecto
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Predecir las etiquetas en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Analizar los resultados
print("Resultados del clasificador SVM con parámetros por defecto:")
print(classification_report(y_test, y_pred))

Resultados del clasificador SVM con parámetros por defecto:
              precision    recall  f1-score   support

       False       0.94      0.99      0.97       455
        True       0.97      0.81      0.88       145

    accuracy                           0.95       600
   macro avg       0.96      0.90      0.92       600
weighted avg       0.95      0.95      0.95       600



In [22]:
# Define una función para tokenizar las URLs
def tokenizar_url(url):
    return url

# Definir la cuadrícula de hiperparámetros a explorar
param_grid = {
    'classifier__C': [0.1, 1, 10, 100], # Parámetro de regularización
    'classifier__kernel': ['linear', 'rbf', 'poly'], # Tipo de kernel
    'classifier__gamma': ['scale', 'auto'], # Coeficiente gamma 
}

# Crear un pipeline con CountVectorizer y SVC
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(analyzer=tokenizar_url)),
    ('classifier', SVC())
])

# Inicializar GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)

# Entrenar GridSearchCV
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:", best_params)

# Obtener la mejor puntuación de validación cruzada
best_score = grid_search.best_score_
print("Mejor puntuación de validación cruzada:", best_score)

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)
print("Resultados del mejor modelo:")
print(classification_report(y_test, y_pred))

Mejores hiperparámetros encontrados: {'classifier__C': 10, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}
Mejor puntuación de validación cruzada: 0.9545728949199722
Resultados del mejor modelo:
              precision    recall  f1-score   support

       False       0.98      0.98      0.98       455
        True       0.92      0.92      0.92       145

    accuracy                           0.96       600
   macro avg       0.95      0.95      0.95       600
weighted avg       0.96      0.96      0.96       600



In [23]:
import joblib
import os

# Especifica la ruta donde deseas guardar el modelo
ruta_modelo = "C:/Users/lenovo/Desktop/Proyectos Machine Learning/Bootcamp/Proyecto de NLP/Model/modelo_svm.joblib"

# Comprueba si la carpeta existe, si no, créala
ruta_carpeta = os.path.dirname(ruta_modelo)
if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)

# Guarda el modelo en el archivo especificado
joblib.dump(pipeline, ruta_modelo)

print("El modelo se ha guardado correctamente en:", ruta_modelo)

El modelo se ha guardado correctamente en: C:/Users/lenovo/Desktop/Proyectos Machine Learning/Bootcamp/Proyecto de NLP/Model/modelo_svm.joblib
